In [1]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [2]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

In [4]:
# Load data 
iris_data = load_iris()
iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris['target'] = iris_data.target
data, labels = iris.iloc[:, :-1].values, iris.iloc[:, -1].values
data = batch_norm(np.array(data, dtype=np.float32))

In [ ]:
def fitness_function(X: np.ndarray, DS: Tuple[np.ndarray, np.ndarray], C) -> np.ndarray:
    """
    Objective function for optimizing Möbius measures:
    - X: 2D array of shape (n_particles, n_mobius_features)
    - DS: Tuple (X_data, y_data)
    - C: Choquet similarity function (e.g., Choquet_classic)
    
    Returns:
    - 1D array of negative LOO accuracy (to minimize)
    """
    results = []
    for mobius_vector in X:
        # Step 1: Convert vector to Möbius measure
        mobius_dict = vector_to_mobius_dict(mobius_vector)

        # Step 2: Derive capacity from Möbius
        capacity_dict = mobius_to_capacity(mobius_dict)

        # Step 3: (Optional) Check monotonicity or normalize capacity
        if not is_monotonic(capacity_dict):
            results.append(float('inf'))  # Penalize non-monotonic capacity
            continue

        # Step 4: Convert capacity_dict to list of Capacity objects if needed
        capacity_list = [
            Capacity(list(k), v) for k, v in capacity_dict.items()
        ]

        # Step 5: Evaluate using fuzzy KNN + LOO
        acc = FuzzLOO(C, DS, capacity_list)

        # Step 6: Use negative accuracy for minimization
        results.append(-acc)

    return np.array(results)
